
## The following code is used for a quick and dirty Tensorflow experimental set up.

Different Cell types can be tested, drop out ratios etc. Tensorboard can be used to track the changes during the training. Also the cell hidden state and and current cell states can be better tracked and manipulated.

In [282]:
import pandas as pd
import numpy as np


from keras import backend as K
from keras.layers import Dense
from keras.objectives import mean_absolute_error




from dateutil.relativedelta import relativedelta
import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session

import itertools
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
#from keras.models import Sequential,Model
#from keras.layers import Input, Dense
#from keras.layers import LSTM, SimpleRNN, GRU,GlobalMaxPooling1D,Conv1D,MaxPooling1D, Dropout, Bidirectional
#from keras import optimizers
from sklearn.metrics import mean_absolute_error

In [284]:
data = pd.read_csv("data/demand_anonymized_20170802.csv", delimiter=";", parse_dates=["Month"])

In [285]:
eval_set = pd.read_csv('data/eval_correct.csv')

In [286]:
series = data.groupby(["SalOrg", "Material", "Month"])["OrderQty"].sum().reset_index()
series = series.sort_values(by=["SalOrg", "Material", "Month"])
series.head()

,SalOrg,Material,Month,OrderQty
0,97LK,00IYcj,2012-05-01,2
1,97LK,00IYcj,2012-06-01,13
2,97LK,00IYcj,2012-07-01,1
3,97LK,00IYcj,2012-09-01,30
4,97LK,00IYcj,2012-11-01,1


In [287]:
eval_comb = eval_set[['Material', 'SalOrg']]
eval_comb = list(set([tuple(x) for x in eval_comb.values])) # gets unique combinations
comb = list(itertools.product(*[eval_comb, list(series['Month'].unique())]))
comb = [(t[0], t[1], m) for t, m in comb]
series2 = pd.DataFrame(comb, columns=['Material', 'SalOrg', 'Month'])
series2 = series2.sort_values(by=['Material', 'SalOrg', 'Month' ])
series2 = series2.merge(series, on=['Month', 'Material', 'SalOrg'], how='left')
series2 = series2.fillna(0)


# validation folds
#training on the last 4 month, since othervise lags will be very sparce (go beyond the availiable data)
def create_validation(data, date, m):
    return data[(data["Month"] >= pd.to_datetime(date) - relativedelta(months=m))&
                (data["Month"] < pd.to_datetime(date))].index, data[(data["Month"] >= pd.to_datetime(date)) &
                                                                    (data["Month"] < pd.to_datetime(date) + relativedelta(months=3))].index

validation_months = ['2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01']

#preparing validation folds
folds = []
for month in validation_months:
    folds.append(create_validation(series2, month, 4))
    
num_lagged = 24
num_predict = 3
def lag_feature(df, colname, lag, num_pred):
    cols_lagged = []
    for i in range(lag, -1, -1):
        col_lagged = '{}(t-{})'.format(colname, i)
        df[col_lagged] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(i)
        cols_lagged.append(col_lagged)
    cols_fut = []
    for i in range(1, num_pred):
        col_fut_name = '{}(t+{})'.format(colname, i)
        df[col_fut_name] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(-i)
        cols_fut.append(col_fut_name)

    return df, cols_lagged, cols_fut

series, lagged, cols_future = lag_feature(series2, 'OrderQty', num_lagged, num_predict)



reframed = series[lagged + cols_future]
reframed.dropna(inplace=True)
idx = reframed.index
values = reframed.values
values = values.astype('float32')


/Users/oleksiyostapenko/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [288]:

# fit the MinMax sacler to normalize features, to unscale later same dimentionality should be used i.e. 27 columns
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = scaler.fit(values)


In [289]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaler_train = scaler.fit(values[:,:-3])
scaler_valid = scaler.fit( np.reshape(values[:, -3], [-1,1]))

In [290]:
fold=folds[0]
train = reframed.loc[fold[0]] 
test = reframed.loc[fold[1]]

In [291]:
train.iloc[:,:-3] = scaler_train.transform(train.iloc[:,:-3])
train.iloc[:,-3] = scaler_valid.transform( np.reshape(np.array(train.iloc[:,-3]), [-1,1]) )

test.iloc[:,:-3] = scaler_train.transform(test.iloc[:,:-3])
test.iloc[:,-3] = scaler_valid.transform(np.reshape(np.array(test.iloc[:,-3]), [-1,1]))


In [292]:
#scaled_train = scaler.transform(train)
#scaled_val = scaler.transform(test)

In [293]:
train_X, train_y = train.iloc[:, :-1 * num_predict], train.iloc[:, -1 * num_predict:]
test_X, test_y = test.iloc[:, :-1 * num_predict], test.iloc[:, -1 * num_predict:]
train_y_ts, test_y_ts = train_X.iloc[:,1::1], test_X.iloc[:,1::1]   

In [294]:
train_y_ts = pd.concat([pd.DataFrame(train_y_ts),pd.DataFrame(train_y.iloc[:,0])], axis=1 )

In [295]:
test_y_ts = pd.concat([pd.DataFrame(test_y_ts), pd.DataFrame(test_y.iloc[:,0])], axis=1)

In [296]:
def fc_layer(input, size_in, size_out, name="fc"):
        with tf.name_scope(name):
            w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
            act = tf.matmul(input, w) + b
            tf.summary.histogram("weights", w)
            tf.summary.histogram("biases", b)
            tf.summary.histogram("activations", act)
            return act, w, b
    

In [297]:
def conv_layer(input, size_in, size_out, name="conv"):
  with tf.name_scope(name):
    kernel = tf.Variable(tf.truncated_normal([1, 3, size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    conv = tf.nn.conv2d(input, kernel, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    #tf.summary.histogram("weights", kernel)
    #tf.summary.histogram("biases", b)
    #tf.summary.histogram("activations", act)
    #return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    return act

## Model Definition

In [303]:
batch_size = 100
LOGDIR = "/Users/oleksiyostapenko/Documents/DSG/Demand Prediction/tmp/logs"
learning_rate = 1E-5

state_size = 10
num_layers = 1
 
tf.reset_default_graph()
sess = tf.Session()
#K.set_session(sess)
# Setup placeholders, and reshape the data
x_train = tf.placeholder(tf.float32, shape=[None, 24], name="x")
x_image = tf.reshape(x_train, [-1, 1, 24, 1])

#x_image = tf.reshape(x_train, [-1, 2, 12]) # -1 stays for readjust the shape, so it would become the batch size
#tf.summary.image('input', x_image, 3)

y = tf.placeholder(tf.float32, shape=[None, 24], name="labels")
#valid_dataset = tf.constant(dtype=tf.float32, value=X_test_fet.as_matrix())
#y_test = tf.constant(dtype=tf.float32, value=labels_test.as_matrix())

#init_state = tf.placeholder(tf.float32, [None, state_size])

########## Conv Layer first ##########

conv_act = conv_layer(x_image, 1, 32, "conv1")
#conv1 = Conv1D(32, 3, padding='valid', activation='relu', strides=1)(x)
#conv_mp = tf.nn.max_pool(conv_act, ksize=[1, 1, 3, 1], strides=[1, 1, 3, 1], padding="SAME")
con_out_size = 24*1*32
flattened = tf.reshape(conv_act, [-1, con_out_size])

######################################


cell_state = tf.placeholder(tf.float32, [num_layers,  None, state_size])
hidden_state = tf.placeholder(tf.float32, [num_layers,  None, state_size])

#multilayered lstn
#state_per_layer_list = tf.unstack(init_state, axis=0)
rnn_tuple_state = tuple(
    [tf.contrib.rnn.LSTMStateTuple(cell_state[idx], hidden_state[idx]) # here once for 
     for idx in range(num_layers)]
)

#inputs_series = tf.unstack(flattened, axis=0)

#W1
#b1 - are in the tf cell

W2 = tf.Variable(np.random.rand(state_size, 1),dtype=tf.float32)
#b2 = tf.Variable(np.zeros((1,1)), dtype=tf.float32)
b2 = tf.Variable(tf.random_normal([1]))

#stacked_rnn = []
#for _ in range(num_layers):
#    stacked_rnn.append(tf.contrib.rnn.LSTMCell(state_size, state_is_tuple=True))

#cellLSTM = tf.contrib.rnn.LSTMCell(state_size, state_is_tuple=True)

cells = []
for _ in range(num_layers):
        cell = tf.contrib.rnn.BasicLSTMCell(state_size, state_is_tuple=True)
        cells.append(cell)
        
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

flattened2 = tf.reshape(flattened, [-1, 24, 32])
#tf.expand_dims(flattened2, -1)

states_series, current_state = tf.nn.dynamic_rnn(cell, flattened2, initial_state=rnn_tuple_state)
states_series = tf.reshape(states_series, [-1, state_size])

logits = tf.matmul(states_series, W2) + b2 
logits_per_period = tf.reshape(logits, [-1, 24])
labels = tf.reshape(y, [-1])


logits_series = tf.unstack(logits_per_period, axis=1)



    
with tf.name_scope("objective"):   
        #loss is the mean absolute error
        losses = tf.abs(tf.subtract(logits,labels))
        total_loss = tf.reduce_mean(losses)
        tf.summary.scalar("total_loss", total_loss)

with tf.name_scope("train"):
        #train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)
        train_step = tf.train.AdagradOptimizer(learning_rate).minimize(total_loss)
        #train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(objective)
        
with tf.name_scope("mae"):
        #correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        mae = tf.metrics.mean_absolute_error(tf.reshape(logits_series,[24,-1])[23],y[:,23])
        #tf.summary.scalar("mae", mae)

summ = tf.summary.merge_all()


    #embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    #assignment = embedding.assign(embedding_input)
saver = tf.train.Saver()

init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
sess.run(init_g)
sess.run(init_l)


writer = tf.summary.FileWriter(LOGDIR)
writer.add_graph(sess.graph)



In [160]:
i=0
_current_cell_state = np.zeros((num_layers, batch_size, state_size))
_current_hidden_state = np.zeros((num_layers, batch_size, state_size))
        
last = i*batch_size
point_X = train_X.iloc[last:last+batch_size,:].as_matrix()
y_label = train_y_ts.iloc[last:last+batch_size,:]

ll = sess.run( logits_series, feed_dict={x_train: point_X, y: y_label,
                                                                   cell_state: _current_cell_state,
                                                                   hidden_state: _current_hidden_state})
#ls = sess.run(logits_series, feed_dict={x_train: point_X, y: y_label, cell_state: _current_cell_state, hidden_state: _current_hidden_state})


## Model Training

In [333]:


#_current_cell_state = np.zeros((batch_size, state_size))  
#_current_hidden_state = np.zeros((batch_size, state_size))           

num_of_batches = train_X.shape[0]/batch_size
num_epochs = 5

for epoch_idx in range(num_epochs):
    _current_cell_state = np.zeros((num_layers, batch_size, state_size))
    _current_hidden_state = np.zeros((num_layers, batch_size, state_size))
    print "epoch", epoch_idx
    for i in range(num_of_batches):
        #_init_state = np.zeros((num_layers, 2, batch_size, state_size))
        _current_cell_state = np.zeros((num_layers, batch_size, state_size))
        _current_hidden_state = np.zeros((num_layers, batch_size, state_size))
        
        last = i*batch_size
        point_X = train_X.iloc[last:last+batch_size,:].as_matrix()
        y_label = train_y_ts.iloc[last:last+batch_size,:]
        
        _total_loss, _train_step, _current_state, error, summary = sess.run(
                [total_loss, train_step, current_state, mae, summ], feed_dict={x_train: point_X, y: y_label,
                                                                   cell_state: _current_cell_state,
                                                                   hidden_state: _current_hidden_state})
        writer.add_summary(summary, i+num_of_batches*epoch_idx)
        
        #_current_cell_state = _current_state.c
        #_current_hidden_state = _current_state.h
            
            
        if i % 500 == 0:
            #MAE only 1 month ahead
            print("Step ",i, "MAE: ", error)
    
    
    _current_cell_state = np.zeros((num_layers, test_y_ts.shape[0], state_size))
    _current_hidden_state = np.zeros((num_layers, test_y_ts.shape[0], state_size))
    #_init_state = np.zeros((num_layers, 2, test_y_ts.shape[0], state_size))d
    
    lg_ser_month_1 = sess.run( logits_series, feed_dict={x_train: test_X.as_matrix(), y: test_y_ts, 
                                              cell_state: _current_cell_state, hidden_state: _current_hidden_state})
    
    #incorporating prediction for the month 1 into the test data to predict month 2
    # snce we are predicting, the labels dont play any role now
    train_y_ts_month_2 = test_X.iloc[:,1:].as_matrix
    train_y_ts_month_2=train_y_ts_month_2.join(pd.DataFrame(lg_ser_month_1[-1]))
    
    lg_ser_month_2 = sess.run( logits_series, feed_dict={x_train: train_y_ts_month_2.as_matrix(), y: test_y_ts, 
                                              cell_state: _current_cell_state,hidden_state: _current_hidden_state})
    
    #incorporating prediction for the month 2 into the test data to predict month 3
    train_y_ts_month_3 = train_y_ts_month_2.iloc[:,1:]
    train_y_ts_month_3[:,23]=lg_ser_month_2[-1]
    
    lg_ser_month_3 = sess.run( logits_series, feed_dict={x_train: train_y_ts_month_3.as_matrix(), y: test_y_ts, 
                                              cell_state: _current_cell_state,hidden_state: _current_hidden_state})
    
    pred = pd.concat([pd.DataFrame(lg_ser_month_1[-1]), pd.DataFrame(lg_ser_month_2[-1]), pd.DataFrame(lg_ser_month_3[-1])], axis=1).as_matrix()
    pred = np.reshape(pred, (test_y_ts.shape[0],1))
    
    y_real = np.reshape(test_y.as_matrix(), ((test_y_ts.shape[0],1))) 
    
    Epoch_Error = mean_absolute_error(y_pred=scaler_valid.inverse_transform(pred), y_true=scaler_valid.inverse_transform(y_real))
    
    print("Error ",Epoch_Error)
            
            #_current_cell_state = _current_state.c
            #_current_hidden_state = _current_state.h
            
            
        #sess.run(assignment, feed_dict={x: mnist.test.images[:1024], y: mnist.test.labels[:1024]})
        #saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)


epoch 0
('Step ', 0, 'MAE: ', (0.18970072, 0.18969749))
('Step ', 500, 'MAE: ', (0.18832652, 0.18832393))
('Step ', 1000, 'MAE: ', (0.18701597, 0.18701315))
('Step ', 1500, 'MAE: ', (0.18577777, 0.18577604))


ValueError: columns overlap but no suffix specified: Index([0], dtype='object')

In [ ]:
sess.close()

In [331]:

train_y_ts_month_2=train_y_ts_month_2.join(pd.DataFrame(lg_ser_month_1[-1]))

In [336]:
train_y_ts_month_2 = test_X.iloc[:,1:].as_matrix()

In [344]:
a = np.append(train_y_ts_month_2, lg_ser_month_2[-1], axis=1) 

ValueError: all the input arrays must have same number of dimensions

In [343]:
a.shape

(2784672,)